In [ ]:
# Pipe Drainage Calculation
import math

# Given data
initial_pressure_barg = 13  # Initial pressure in the pipes (barg)
outlet_pressure_barg = 10  # Pressure at the outlet (barg)
pump_pressure_barg = 10  # Pump pressure (barg)
pump_flow_lpm = 50  # Pump flow rate (LPM)
outflow_lpm = 426  # Outflow rate at the outlet (LPM)

# Pipe dimensions (diameter in mm, length in m)
pipes = [
    (80, 0.634),
    (100, 49.8),
    (100, 144.4),
    (100, 136.7),
    (100, 58.6)
]

# Convert diameter to meters and calculate volume of each pipe
def calculate_pipe_volume(diameter_mm, length_m):
    radius_m = (diameter_mm / 1000) / 2  # Convert to meters and find radius
    volume_m3 = math.pi * (radius_m ** 2) * length_m  # Cylinder volume formula
    return volume_m3

# Calculate volumes for each pipe
pipe_volumes = [calculate_pipe_volume(d, l) for d, l in pipes]

# Sum total volume in cubic meters
total_volume_m3 = sum(pipe_volumes)

# Convert total volume from cubic meters to liters
total_volume_liters = total_volume_m3 * 1000  # 1 m³ = 1000 liters

# Net flow rate (outflow - inflow)
net_outflow_lpm = outflow_lpm - pump_flow_lpm

# Apply a pressure adjustment factor based on the pressure difference
pressure_ratio = (initial_pressure_barg - outlet_pressure_barg) / pump_pressure_barg
adjusted_net_outflow_lpm = net_outflow_lpm * (1 + pressure_ratio)

# Calculate time required to empty the pipe system (in minutes)
time_minutes = total_volume_liters / adjusted_net_outflow_lpm

# Convert time to seconds
time_seconds = time_minutes * 60  # 1 minute = 60 seconds

# Output the updated time required for the system to be drained
time_seconds
